# Predict "Hoax" With Logistic Regression

- There was a typo and a feature slipped through that wasn't supposed to
    - ➜ fixed
    - The sets are the same now, but the phenomenon is remaining.
- Maybe it's because the intercept was scaled to zero?  
    - ➜ Try only scaling the original floats.
    - Score improved to .54/.5 , but unscaled remains .6
- Look at (unscaled_coeff / $\sigma$) to understand feature importance
    - ➜ Try repeating the regression with these features only
    - Score improved to .55/.5 , and unscaled remains .58
- Checked Lasso (with all features)
    - ➜ same results as Ridge: .54/.5
- Relaxed my limitation of C,
    - ➜ improved to .57/.5 with C=6
- Ran logistic regression on unscaled features with no regularization
    - ➜ improved to .61/.5
    - Far out-performed the paper-author's models
- Let's try SFS with scaled features
    - Success!
- Conclusion:
    - The unscaled data was causing regularization to do feature selection in a better way.
    - When the feature selection was improved with 

### When to use sklearn's GridSearchCV to wrap SFS?

- Technical particularity:
    - It's only possible to get access to the SFS instance for the param combination that GridSearchCV found to be optimal.
    - This mitigates output complexity, but prevents thorough review.
    - It would prevent a review of both SFFS and SFBS within the same gridsearch as is done here, for instance.
- Pros
    - GridSearchCV is great for automated production applications, to make the code robust, concise, and palatable for other users.
- Cons
    - GridSearchCV wouldn't let me do a viz overview of the param_grid outputs.
    - (it's grid ***search***, not grid ***exploration***.) 

#### Previous thoughts on why not to use sklearn's GridSearchCV with SFS:

## Get hoax data

### imports

In [1]:
import os, re, patsy
import pandas as pd, numpy as np
from datetime import datetime as dt
from sklearn.model_selection import train_test_split
path = '/home/bhrdwj/git/predwikt/data/raw/wiki_reliability/unzipped/'

In [2]:
feature_data_raw = (pd.read_csv(path+'hoax_features.csv', usecols=lambda x: x not in ['Unnamed: 0'])
       .rename(columns={'headings_by_level(2)':'headings_by_level_2', 'revision_id.key':'revision_id_key'}))

### train test split

- Dataset has paired observations: one for positive and one for negative for particular articles.
- Train-test-split needs to keep paired observations in the same splits, for useful training to happen 

#### Organize paired data by making **indexing series**

- Indexing series ≡
    - index: revision_id
    - value: revision_id_key
- Make indexing series for both:
    - hoaxes: positive
    - not hoaxes: negative

In [28]:
feature_data_raw.has_template.value_counts()

1.0    1398
0.0    1390
Name: has_template, dtype: int64

- Note there are eight excess observations of positive (hoax) revs:
    - therefore, perform train-test split based on non-hoax first,
    - then use indices of that split to split hoaxes accordingly

In [29]:
df = feature_data_raw[['revision_id', 'revision_id_key', 'has_template']]
revid_ser_neg = df.copy().loc[df.has_template==0].set_index('revision_id')['revision_id_key']
revid_ser_pos = df.copy().loc[df.has_template==1].set_index('revision_id')['revision_id_key']
del df

#### Train-test split

Split the negative revids (indices)  
Get positive indices from revision_id_key (values)

In [30]:
revid_neg_tr, revid_neg_te = train_test_split(revid_ser_neg, test_size=.2, random_state=0)
revid_pos_tr = revid_ser_pos.loc[revid_neg_tr.values]
revid_pos_te = revid_ser_pos.loc[revid_neg_te.values]

Reconcatenate neg and pos observations into train and test revision_id indices

In [31]:
revid_tr = pd.concat((revid_pos_tr, revid_neg_tr))
revid_te = pd.concat((revid_pos_te, revid_neg_te))

Select original dataset into train and test using revision_id

In [32]:
fea_revid = feature_data_raw.set_index('revision_id')
dftr = fea_revid.loc[revid_tr.index].dropna()
dfte = fea_revid.loc[revid_te.index].dropna()

Clean namespace and view data summary

In [33]:
del revid_ser_neg, revid_ser_pos
del revid_neg_tr, revid_neg_te, revid_pos_tr, revid_pos_te
del revid_tr, revid_te, fea_revid

In [34]:
dftr[dftr.columns.difference(['page_id','revision_id_key','has_template'])].describe().T.sort_values(by='mean');

### prep

##### Separate X and y, remove non-features, onehotify categoricals

In [35]:
ytr = dftr.has_template
Xtr = dftr[dftr.columns.difference(['page_id','revision_id_key','has_template'])]
Xtr = patsy.dmatrix('~ '+' + '.join(Xtr.columns), data=Xtr, NA_action='drop', return_type='dataframe')

yte = dfte.has_template
Xte = dfte[dfte.columns.difference(['page_id','revision_id_key','has_template'])]
Xte = patsy.dmatrix('~ '+' + '.join(Xte.columns), data=Xte, NA_action='drop', return_type='dataframe')

##### Make complete list of features in case the test (or train) set doesn't include any of a rare class

In [36]:
Xcols = list(
    set(Xtr.columns.tolist())
    .union(set(Xte.columns.tolist()))
)

for col in Xcols:
    if col not in Xte:
        Xte[col] = 0
    if col not in Xtr:
        Xtr[col] = 0

##### Reindex with the complete feature list

In [37]:
Xtr = Xtr.reindex(columns=Xcols)
Xte = Xte.reindex(columns=Xcols)
Xtr.shape, Xte.shape

((2221, 25), (556, 25))

## Fit model

### imports

In [40]:
from sklearn.model_selection import ParameterGrid, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

### functions

##### get_fitfeats_sfs_pipe

In [42]:
def bh_sfs_gridsearch(Xtr, ytr, pipe, param_grid:dict):
    """
    Concept:
        Like sklearn's GridSearch, but relying on sfs for cross-validation.
        Returns all feature sets and cv scores from SFFS and SFBS runs and sub-runs for each params combo.
        (sklearn's GridSearch only returns model outputs from its winning param combo)
    Args:
        Xtr: pd.DataFrame of features
        ytr: pd.Series target
        pipe: sklearn Pipeline ending in a mlxtend SequentialFeatureSelector instance.
        param_grid: dict of lists, params of pipe, input for sklearn ParameterGrid
    Returns:
        dict:
        - keys are numbers starting in 1, for each cell of ParameterGrid
        - values are lists of feature names
    """
    print(f'start time: {dt.now()}')
    pg = {i:j for i,j in enumerate(ParameterGrid(param_grid), start=1)}
    sfs_featdict = {}
    sfs_scoredict = {}
    for i in pg:
        pipe.set_params(**pg[i]).fit(Xtr, ytr)
        k_feats = pg[i]['sfs__k_features']
        idx_tup = pipe.steps[-1][1].get_metric_dict()[k_feats]['feature_idx']
        
        d = {k:list(v['feature_idx']) for k,v in pipe.steps[1][1].subsets_.items()}  # IMPROVEMENT: change pipe.steps[1] to directly look for the sfs step
        for j in d: d[j] = Xtr.columns[d[j]]
        sfs_featdict[i] = d
        
        s = {k:v['avg_score'] for k,v in pipe.steps[1][1].subsets_.items()} # IMPROVEMENT: change pipe.steps[1] to directly look for the sfs step
        sfs_scoredict[i] = s
    print(f'finish time: {dt.now()}')
    return sfs_featdict, sfs_scoredict

### instances

In [43]:
scaler = StandardScaler()
lr_sfs = LogisticRegression(penalty='l2', max_iter=1000, fit_intercept=True)
cv_sfs = StratifiedKFold(n_splits=5, shuffle=False)
sfs = SFS(estimator=lr_sfs, forward=True, floating=False, scoring='accuracy', cv=cv_sfs)
sffs = SFS(estimator=lr_sfs, forward=True, floating=True, scoring='accuracy', cv=cv_sfs)
sfbs = SFS(estimator=lr_sfs, forward=False, floating=True, scoring='accuracy', cv=cv_sfs)
sfs_pipe = Pipeline([('scaler', scaler),('sfs', sfs)])

### fit

#### sffs

In [44]:
sfs_pipe_param_grid = {
    'sfs': [sffs, sfbs],
    'sfs__k_features': [1, len(Xtr.columns)],
    'sfs__estimator': [lr_sfs],
    'sfs__estimator__C': [.013]
}

sfs_featdict, sfs_scoredict = bh_sfs_gridsearch(Xtr, ytr, sfs_pipe, sfs_pipe_param_grid)

start time: 2022-01-14 16:17:47.109755
finish time: 2022-01-14 16:20:34.958287


#### pickle

In [ ]:
import pickle

with open('../data/processed/sfs_scoredict.StratifiedKFoldkle','wb+') as f:
    pickle.dump(sfs_scoredict, f)
with open('../data/processed/sfs_featdict.pickle','wb+') as f:
    pickle.dump(sfs_featdict, f)

# with open('../data/processed/sfs_featdict.pickle','rb') as f:
#     sfs_featdict = pickle.load(f)
# with open('../data/processed/sfs_scoredict.pickle','rb') as f:
#     sfs_scoredict = pickle.load(f)

## Manage model output

### extract feats

#### get onehot df ~ features selected by sfs

##### munge_onehotdf_from_sfs_featdict (function)

In [ ]:
def munge_onehotdf_from_sfs_featdict(sfs_featdict):
    
    # simple 1D list of sfs  feats
    sfs_feat_set = set()
    for i in sfs_featdict:              # runs
        for j in sfs_featdict[i]:       # kfeats
            sfs_feats = list(sfs_feat_set.union(set(sfs_featdict[i][j])))
    
    # make multiindex columns and empty dataframe
    col = pd.MultiIndex.from_tuples(
        [(j,k) for j in sfs_featdict for k in sfs_featdict[j]])
    idx = pd.Index(sfs_feats, name='feature')
    df = pd.DataFrame('-', idx, col)
    
    # insert onehots into df
    for i in sfs_feats:                           # feats
        for j in sfs_featdict:                        # runs
            for k in sfs_featdict[j]:             # steps within run
                df.loc[i,(j,k)] = int(i in sfs_featdict[j][k])
    
    df = df.astype(int) 
    return df

##### get df

In [ ]:
sfs_onehots = munge_onehotdf_from_sfs_featdict(sfs_featdict)

#### get simple 1D lists of sfs runs, kfeats, and feats

In [ ]:
sfs_runs = list(sfs_featdict.keys())

sfs_feat_set, sfs_kfeats_set = set(), set()
for i in sfs_featdict:              # runs
    for j in sfs_featdict[i]:       # kfeats
        sfs_feats = list(sfs_feat_set.union(set(sfs_featdict[i][j])))
        sfs_kfeats = list(sfs_kfeats_set.union(set([j])))

#### sort the index of features by occurence

##### count occurrences of features to sort the index

In [ ]:
idx = pd.IndexSlice
sfs_feat_usecounts = sfs_onehots.loc[:,idx[1,:]].sum(axis=1)
for i in sfs_featdict:
    sfs_feat_usecounts += sfs_onehots.loc[:,idx[i,:]].sum(axis=1)

##### sort features by frequency of occurence in sfs cases

In [ ]:
sfs_onehots = sfs_onehots.reindex(index=sfs_feat_usecounts.sort_values(ascending=False).index)
sfs_onehots = sfs_onehots.sort_index(axis=1)

## Viz

##### imports

In [ ]:
import matplotlib
%matplotlib inline
matplotlib.use('TkAgg')

import matplotlib.pyplot as plt
import seaborn as sns

#### plot feature selection

In [ ]:
feat_fig, ax = plt.subplots(1,2, figsize=(16,8))
feat_fig.subplots_adjust(wspace=.5)

(sns.heatmap(
    sfs_onehots.loc[:,idx[2,:]].T.droplevel(level=0).T, 
    ax=ax[0], cbar=False)
    .set(ylabel=None, xlabel='kfeats')   
)
(sns.heatmap(
    sfs_onehots.loc[:,idx[3,:]].T.droplevel(level=0).T, 
    ax=ax[1], cbar=False)
    .set(ylabel=None, xlabel='kfeats')
)

ax[0].set_title('Sequential Floating Forward Selection')
ax[1].set_title('Sequential Floating Backward Selection')
feat_fig.suptitle('Logistic Regression: Features selected, graphed against k_features')
feat_fig.show()

#### plot cv scores

In [ ]:
scor_fig, ax = plt.subplots(1,2, figsize=(14,4))
scor_fig.subplots_adjust(wspace=.4)

pd.Series(sfs_scoredict[2]).plot(ax=ax[0])
pd.Series(sfs_scoredict[3]).plot(ax=ax[1])
for axis in ax:
    axis.set_xlabel('kfeats')
    axis.set_ylabel('accuracy')
ax[0].set_title('Sequential Floating Forward Selection')
ax[1].set_title('Sequential Floating Backward Selection')
scor_fig.suptitle('Logistic Regression: Cross-validation accuracy score from SFS (baseline accuracy 0.5)')
scor_fig.show()

### Get top 5 feats sets by cross-val score

##### create empty dataframe for scores

In [ ]:
df_scores = pd.DataFrame('-', ['score'], sfs_onehots.columns)

##### insert scores into dataframe

In [ ]:
for j in sfs_runs:                        # runs
    for k in sfs_featdict[j]:             # steps within run
        df_scores.loc['score',(j,k)] = sfs_scoredict[j][k]   
df_scores = df_scores.sort_index(axis=1)

##### select the 5 multiindices that had the best cross-val scores across sffs and sfbs

##### cols_5best = df_scores.T.astype(float).nlargest(5, columns='score').T.columns
scores_5best = df_scores.T.astype(float).nlargest(5, columns='score').values.tolist()

##### use the indices to make a dataframe of onehots; then get lists of features

In [ ]:
feats_5best_onehots = sfs_onehots[cols_5best].T.droplevel(level=0).reset_index(drop=True).T
feats_5best = {}
indices_5best = []
for i in feats_5best_onehots.columns:
    feats_5best[i] = set(feats_5best_onehots[i].loc[lambda x: x>0].index.tolist())

## Tune hyperparameters

In [ ]:
sfs_featdict[2][1]

In [ ]:
test_scores = {}
best_Cs = {}
for i in sfs_featdict:             # runs
    test_scores[i] = {}
    best_Cs[i] = {}
    for j in sfs_featdict[i]:       # steps
        lr_C = LogisticRegressionCV(penalty='l2', Cs=[.01,.1,1,10,100], max_iter=1000, fit_intercept=True)
        lr_C.fit(Xtr[sfs_featdict[i][j]], ytr)
        best_Cs[i][j] = lr_C.C_[0]
        test_scores[i][j] = lr_C.score(Xte[sfs_featdict[i][j]], yte)

#### Plot test scores

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(14,4))
fig.subplots_adjust(wspace=.4)

pd.Series(test_scores[2]).plot(ax=ax[0])
pd.Series(test_scores[3]).plot(ax=ax[1])
for axis in ax:
    axis.set_xlabel('kfeats')
    axis.set_ylabel('accuracy')
ax[0].set_title('Sequential Floating Forward Selection')
ax[1].set_title('Sequential Floating Backward Selection')
fig.suptitle('Logistic Regression: Test-set accuracy scores for each feature set (baseline accuracy 0.5)')
fig.show()

In [ ]:
fig

#### Plot optimal C from LogisticRegressionCV

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(14,4))
fig.subplots_adjust(wspace=.4)

pd.Series(best_Cs[2]).plot(ax=ax[0])
pd.Series(best_Cs[3]).plot(ax=ax[1])
for axis in ax:
    axis.set_xlabel('kfeats')
    axis.set_ylabel('accuracy')
    axis.set_yscale('log')
ax[0].set_title('Sequential Floating Forward Selection')
ax[1].set_title('Sequential Floating Backward Selection')
fig.suptitle('Logistic Regression: Validation-set accuracy scores for each feature set (baseline accuracy 0.5)')
fig.show()

In [ ]:
feat_fig

In [ ]:
scor_fig

In [ ]:
best_Cs; # all best_Cs

In [ ]:
test_scores; # all scores on test data

In [ ]:
scores_5best # best 5 cross-validation scores

In [ ]:
(pd.DataFrame(test_scores)
 .T
 .stack()
 .loc[cols_5best]
 .sort_index()
 .reset_index()
 .rename(columns={'level_0':'direction', 'level_1':'kfeats', 0:'test_accuracy'})
 .assign(direction=lambda x: x.direction.replace({2:'forward', 3:'backward'}))
 .assign(test_accuracy=lambda x: x.test_accuracy.round(3).astype(str))
 .style.set_caption("Best 5 Feature Selections by cv_score")
)

In [ ]:
feats_5best_onehots[0][feats_5best_onehots[0] > 0].index.tolist()

In [ ]:
list(map(len,feats_5best.values()))

### Choose a feature set

The 10-feature set had the best cross-val score and almost the best test score.  
Its features are also a strict subset of three other top-5 sets, which may suggest lower variance.
It was also optimized with a moderate regularization penalty.

Choose the 10 feature set, and C=1.0. 

## Conclusions

- The smooth rising and falling of cross-val scores suggests a consistent bias-variance tradeoff
- It's useful to see the curves for forwards and backwards, and the 